In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

import torch
import matplotlib.pyplot as plt
from src import models, data
from tqdm.auto import tqdm
import json
import os
import numpy as np
import copy


In [3]:
mt = models.load_model("gptj", fp16=True, device="cuda")

In [22]:
#####################################
relation_name = "person occupation"
#####################################

In [23]:
dataset = data.load_dataset()
relation = dataset.filter(
    relation_names = [relation_name]
)[0]

In [24]:
from src.utils.sweep_utils import read_sweep_results, relation_from_dict

In [25]:
sweep_dict = read_sweep_results(
    "../results/sweep-24-trials/gptj", 
    relation_names=[relation_name], 
    economy=False
)

In [26]:
relation_result = relation_from_dict(sweep_dict[relation_name])

In [27]:
trial_options = list(range(len(relation_result.trials)))
print(f"{trial_options=}")

layer_options = [layer.layer for layer in relation_result.trials[0].layers]
print(f"{layer_options=}")

rank_options = [rank.rank for rank in relation_result.trials[0].layers[0].result.ranks]
print(f"{rank_options=}")

trial_options=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
layer_options=['emb', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
rank_options=[0, 8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 104, 112, 120, 128, 136, 144, 152, 160, 168, 176, 184, 192, 200, 208, 216, 224, 232, 240, 248, 256, 264, 272, 280, 288, 296, 304, 312]


In [28]:
relation_result.best_by_efficacy(beta = 2.25).__dict__

{'layer': 8,
 'beta': AggregateMetric(mean=2.25, stdev=0.0, stderr=0.0, values=[2.25, 2.25, 2.25, 2.25, 2.25, 2.25, 2.25, 2.25, 2.25, 2.25, 2.25, 2.25, 2.25, 2.25, 2.25, 2.25, 2.25, 2.25, 2.25, 2.25, 2.25, 2.25, 2.25, 2.25]),
 'recall': AggregateMetric(mean=0.4944704266017017, stdev=0.07817291710101862, stderr=0.015956979883532067, values=[0.47019867549668876, 0.6115107913669064, 0.39634146341463417, 0.5294117647058824, 0.4968152866242038, 0.5032258064516129, 0.45454545454545453, 0.5172413793103449, 0.5130434782608696, 0.5779816513761468, 0.4470588235294118, 0.46987951807228917, 0.43157894736842106, 0.49645390070921985, 0.6238532110091743, 0.6517857142857143, 0.3763440860215054, 0.38345864661654133, 0.48936170212765956, 0.5826086956521739, 0.4460431654676259, 0.34554973821989526, 0.5656565656565656, 0.4873417721518987]),
 'rank': AggregateMetric(mean=131.66666666666666, stdev=55.520766885513716, stderr=11.333129083126817, values=[168, 48, 152, 120, 152, 72, 104, 96, 80, 112, 96, 64, 72

In [29]:
#########################################################################################################
TRIAL_NO = 7
RANK = 136
LAYER = 8
#########################################################################################################

In [30]:
layer_result = [layer for layer in relation_result.trials[TRIAL_NO].layers if layer.layer == LAYER][0]
rank_result = [rank for rank in layer_result.result.ranks if rank.rank == RANK][0]
rank_result

SweepRankResults(rank=136, efficacy=[0.6293103448275862, 0.7931034482758621, 0.853448275862069], efficacy_successes=[EfficacyTestPair(source=RelationSample(subject='Adelaide Crapsey', object='poet'), target=RelationSample(subject='Augustin-Jean Fresnel', object='physicist')), EfficacyTestPair(source=RelationSample(subject='Anders Fogh Rasmussen', object='politician'), target=RelationSample(subject='Werner Heisenberg', object='physicist')), EfficacyTestPair(source=RelationSample(subject='Andrew Ross Sorkin', object='journalist'), target=RelationSample(subject='Edward Burtynsky', object='photographer')), EfficacyTestPair(source=RelationSample(subject='Antoine Augustin Cournot', object='mathematician'), target=RelationSample(subject='Oskar Lafontaine', object='politician')), EfficacyTestPair(source=RelationSample(subject='Anton Walter', object='composer'), target=RelationSample(subject='Lim Chin Siong', object='politician')), EfficacyTestPair(source=RelationSample(subject='Augustin-Jean F

In [31]:
efficacy_successes = {s.target.subject : s for s in rank_result.efficacy_successes}

for beta_result in layer_result.result.betas:
    faithfulness_successes = beta_result.faithfulness_successes
    for sample in faithfulness_successes:
        if(sample.subject in efficacy_successes):
            print(f"Edit: {efficacy_successes[sample.subject].source} <to> {efficacy_successes[sample.subject].target} -- found in beta: {beta_result.beta}")
            efficacy_successes.pop(sample.subject)
        

Edit: David Gascoyne -> poet <to> Andrew Ross Sorkin -> journalist -- found in beta: 0.5
Edit: John Crewe, 1st Baron Crewe -> politician <to> David Weigel -> journalist -- found in beta: 0.5
Edit: Oskar Lafontaine -> politician <to> Emmanuel Chabrier -> composer -- found in beta: 0.5
Edit: Nureddin Pasha -> politician <to> Enrique Granados -> composer -- found in beta: 0.5
Edit: Naomi Shihab Nye -> poet <to> Felix Salmon -> journalist -- found in beta: 0.5
Edit: Peter Arnett -> journalist <to> James Elroy Flecker -> poet -- found in beta: 0.5
Edit: Thomas Troelsen -> composer <to> Muhammad Iqbal -> poet -- found in beta: 0.5
Edit: William Jennings Bryan -> politician <to> Nikolai Myaskovsky -> composer -- found in beta: 0.5
Edit: Benedict Calvert, 4th Baron Baltimore -> politician <to> Nikolaj Frederik Severin Grundtvig -> poet -- found in beta: 0.5
Edit: Johannes Nucius -> composer <to> Peter Arnett -> journalist -- found in beta: 0.5
Edit: Louis Moreau Gottschalk -> composer <to> Cra

In [32]:
print("No target match found in faithfulness successes for the following:")
for sample in efficacy_successes.values():
    print(f"Edit: {sample.source} <to> {sample.target}")

No target match found in faithfulness successes for the following:
Edit: Jan Tennant -> journalist <to> Edward Burtynsky -> photographer
Edit: Anton Walter -> composer <to> Lim Chin Siong -> politician
Edit: John Horgan -> journalist <to> Cledwyn Hughes, Baron Cledwyn of Penrhos -> politician
Edit: Evan Bayh -> politician <to> Mia Freedman -> journalist
Edit: J. Gwyn Griffiths -> poet <to> Anton Walter -> composer
Edit: Jourdan Miller -> model <to> Karl Taylor Compton -> physicist
Edit: Thomas de Waal -> journalist <to> Hilary Putnam -> philosopher
Edit: Michael William Balfe -> composer <to> J. Anthony Lukas -> journalist
Edit: Ross Garnaut -> economist <to> John Crewe, 1st Baron Crewe -> politician
Edit: Roberto Gerhard -> composer <to> James E. Ferguson -> politician
Edit: Walter Hines Page -> journalist <to> Malcolm MacDonald -> politician


In [34]:
train_samples = layer_result.result.samples
train_samples

[RelationSample(subject='Tony Pua', object='politician'),
 RelationSample(subject='Janet Gunn', object='actor'),
 RelationSample(subject='Valentino Bucchi', object='composer'),
 RelationSample(subject='Christopher Guest', object='comedian'),
 RelationSample(subject='Giovanni Battista Guarini', object='poet'),
 RelationSample(subject='Menachem Mendel Schneerson', object='rabbi'),
 RelationSample(subject='Cristina Peri Rossi', object='novelist'),
 RelationSample(subject='Nils Strindberg', object='photographer')]

In [35]:
prompt_template = relation_result.trials[TRIAL_NO].prompt_template
prompt_template

'{} works as a'

In [36]:
from src import functional, operators, editors

In [37]:
estimator = operators.JacobianIclMeanEstimator(
    mt = mt,
    h_layer = LAYER,
)

operator = estimator(
    relation.set(
        samples = train_samples,
        prompt_templates = [prompt_template],
    )
)

In [42]:
############################################
subject = "Malcolm MacDonald"
############################################

In [43]:
# model predicts correctly
functional.predict_next_token(
    mt = mt,
    prompt = operator.prompt_template.format(subject)
)

[[PredictedToken(token=' politician', prob=0.13466356694698334),
  PredictedToken(token=' journalist', prob=0.11699773371219635),
  PredictedToken(token=' writer', prob=0.06262437254190445),
  PredictedToken(token=' musician', prob=0.04139218106865883),
  PredictedToken(token=' historian', prob=0.028448402881622314)]]

In [44]:
# LRE fails (low faithfulness)
operator(subject=subject, k = 5)

LinearRelationOutput(predictions=[PredictedToken(token=' writer', prob=0.11166161298751831), PredictedToken(token=' poet', prob=0.09550923854112625), PredictedToken(token=' musician', prob=0.0816933736205101), PredictedToken(token=' journalist', prob=0.06462478637695312), PredictedToken(token=' politician', prob=0.05527650564908981)], h=tensor([[ 0.0685,  0.7666,  0.0354,  ...,  2.1816, -1.7412, -0.4724]],
       device='cuda:0', dtype=torch.float16), z=tensor([[-1.7080, -5.2656,  0.7314,  ..., -5.3125,  0.2017,  5.3555]],
       device='cuda:0', dtype=torch.float16, grad_fn=<AddBackward0>))

In [45]:
svd = torch.svd(operator.weight.float())
editor = editors.LowRankPInvEditor(
    lre = operator,
    rank = rank_result.rank,
    n_samples=1, n_new_tokens=1,
    svd = svd
)

In [46]:
efficacy_test_pair = efficacy_successes[subject]
f"Editing: {efficacy_test_pair.source} <to> {efficacy_test_pair.target}"

'Editing: Walter Hines Page -> journalist <to> Malcolm MacDonald -> politician'

In [47]:
# editing succeeds (high efficacy)
editor(
    subject = efficacy_test_pair.source.subject,
    target = efficacy_test_pair.target.subject,
)

LinearRelationEditResult(predicted_tokens=[PredictedToken(token=' politician', prob=0.13382403552532196), PredictedToken(token=' musician', prob=0.09489544481039047), PredictedToken(token=' writer', prob=0.08506372570991516), PredictedToken(token=' journalist', prob=0.07684867829084396), PredictedToken(token=' poet', prob=0.04588843882083893)], model_logits=tensor([-inf, -inf, -inf,  ..., -inf, -inf, -inf], device='cuda:0'), model_generations=['Tony Pua works as a politician\nJanet Gunn works as a actor\nValentino Bucchi works as a composer\nChristopher Guest works as a comedian\nGiovanni Battista Guarini works as a poet\nMenachem Mendel Schneerson works as a rabbi\nCristina Peri Rossi works as a novelist\nNils Strindberg works as a photographer\nWalter Hines Page works as a historian'])